In [2]:
import cv2 as cv
import os
import numpy as np 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow import keras
from tensorflow.keras.utils import to_categorical


def showImage(img,title='Resized Window'):

    #define the screen resulation
    screen_res = 1280, 720
    scale_width = screen_res[0] / img.shape[1]
    scale_height = screen_res[1] / img.shape[0]
    scale = min(scale_width, scale_height)

    #resized window width and height
    window_width = int(img.shape[1] * scale)
    window_height = int(img.shape[0] * scale)

    #cv.WINDOW_NORMAL makes the output window resizealbe
    cv.namedWindow(title, cv.WINDOW_NORMAL)

    #resize the window according to the screen resolution
    cv.resizeWindow(title, window_width, window_height)

    cv.imshow(title, img)
    cv.waitKey(0)
    cv.destroyAllWindows()

import random
#function to get a random file
def random_file(path):
    if(os.path.isfile(path)):
        return path
    else:
        x = os.listdir(path)
        return random_file(os.path.join(path,x[random.randint(0, len(x)-1)]))

#function to get random picture of specific number
def get_rand_pic_of(number):
    x = os.listdir("Dataset")
    y = x[random.randint(0, len(x)-1)]
    x = os.path.join("Dataset",y)
    x = os.path.join(x,str(number))
    return random_file(x)

#methods
#open image in opencv//done
#apply thresholding to image and resize//done
# and save the image in a new folder
#return an array of all the images for a given number and purpose (test or train)
import cv2 as cv
import os
import numpy as np

def open_image(path):
    x = cv.imread(path,0)
    if (x is None):
        raise Exception("File not found")
    return x

def preprocess(image):
    if(np.max(image)<100):
        image = image*255
    image =  cv.resize(image, (32,32), interpolation=cv.INTER_AREA)
    image =  cv.adaptiveThreshold(image,255,cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY,15,10)
    image = image/255

    return image

def one_hot_encoder(num, num_classes = 9):
    x = np.zeros(num_classes)
    x[int(num)] = 1.0
    return x


class ImageRetriever:
    id = "" #file name
    label = "" #0-9
    purpose = "" #training_data or testing_data
    raw_image = None

    def __init__(self, raw_image = None, path = "", purpose = "", id = "")-> None:
        self.raw_image = raw_image
        self.path = path
        self.purpose = purpose
        self.id = id

    

    def get(self, path):
        self.raw_image = preprocess(open_image(path))
        self.id = path.split('/')[-1].split('.')[0]
        self.label = path.split('/')[-2]
        self.label = to_categorical(int(self.label)-1, num_classes=9)
        self.purpose = path.split('/')[1]

def get_data_split(file="training_data", image_retriever = ImageRetriever()):
    path = os.path.join("Dataset",file)
    train = np.ndarray((0,32,32,1))
    labels = np.ndarray((0,10))
    for number_folder in os.listdir(path):
        for number_image in os.listdir(os.path.join(path,number_folder)):
            image_path = os.path.join(path,number_folder,number_image)
            image_retriever.get(image_path)
            train = np.vstack([train,image_retriever.raw_image.reshape((1,32,32,1))])
            labels = np.vstack([labels, image_retriever.label.reshape((1,10))])
    return train, labels

def build_model():
    num_classes = 10

    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3,3),
                    activation='relu',
                    input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    print(model.summary())
    
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer="Adam",
              metrics=['accuracy'])

    return model

def build_and_train_model():
    model = build_model()
    train,labels = get_data_split()
    model.fit(train,labels, epochs = 5)

    test, test_labels = get_data_split(file = "testing_data")

    print(model.evaluate(test,test_labels))
    return model

In [4]:
x = ImageRetriever('Dataset/training_data/4/44516.png')
y = x.get('Dataset/training_data/4/44516.png')


#showImage(x.raw_image)

In [5]:
y = random_file("Dataset")
x.get(y)

#showImage(x.raw_image)
print(f"file is {y}")

file is Dataset/training_data/7/50387.png


In [6]:
x.raw_image.shape


(32, 32)

In [8]:
showImage(x.raw_image)

In [6]:
from tensorflow.keras.datasets import mnist
import tensorflow as tf

In [21]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = cv.bitwise_not(X_train)
X_test = cv.bitwise_not(X_test)

X_train = np.expand_dims(X_train, axis=-1)
X_train = tf.image.resize(X_train, [32,32], method='nearest')
X_test = np.expand_dims(X_test, axis=-1)
X_test = tf.image.resize(X_test, [32,32], method='nearest')


In [4]:
import cv2 as cv

In [16]:

X_train = np.expand_dims(X_train, axis=-1)
X_train = tf.image.resize(X_train, [32,32], method='nearest')


In [17]:
showImage(np.array(X_train[0]))

In [84]:
cv.resize(x,(4,4),cv.INTER_AREA)

array([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,  14,   1, 154,
         253,  90,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0, 249, 253, 249,  64,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0, 136, 253, 253, 253, 212, 135, 132,  16,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0]],

       [[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,  23,  60, 179,
         253, 112,   3,  19,  30,   5,   0,   0,  63,  61,  13,   0,
           0,   0,  

In [28]:
train_filter = np.where((Y_train != 0 ))
test_filter = np.where((Y_test != 0))
X_train, Y_train = X_train[train_filter], Y_train[train_filter]
X_test, Y_test = X_test[test_filter], Y_test[test_filter]



In [29]:
X_train = X_train.reshape((X_train.shape[0], 32, 32, 1))
X_test = X_test.reshape((X_test.shape[0], 32, 32, 1))

ValueError: cannot reshape array of size 42396368 into shape (54077,32,32,1)

In [20]:
# one hot encode target values
Y_train = to_categorical(Y_train-1,num_classes=9)
Y_test = to_categorical(Y_test-1,num_classes=9)

In [22]:

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [23]:
X_train,X_test = prep_pixels(X_train,X_test)

In [25]:
import tensorflow as tf
model = tf.keras.models.load_model('saved_model/my_model')

In [26]:
history = model.fit(X_train,Y_train,batch_size=32,epochs=10)

Epoch 1/10


ValueError: in user code:

    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/alex/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {

In [201]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


def build_model():
    num_classes = 9

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5),
                    activation='relu',
                    input_shape=(32,32,1)))
    model.add(MaxPooling2D(pool_size=(2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2)))
    model.add(Flatten())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    print(model.summary())
    
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer="Adam",
              metrics=['accuracy'])

    return model

def build_and_train_model():
    model = build_model()
    train,labels = get_data_split()
    model.fit(train,labels, epochs = 5)

    test, test_labels = get_data_split(file = "testing_data")

    print(model.evaluate(test,test_labels))
    return model
    
    

In [202]:
from tensorflow import keras

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer="Adam",
              metrics=['accuracy'])


In [168]:
train = np.ndarray((0,32,32,1))
labels = np.ndarray((0,10))

In [171]:
train  = np.vstack([train, x.raw_image.reshape((1,32,32,1))])
labels = np.vstack([labels, x.label.reshape((1,10))])



In [164]:
x.get(get_rand_pic_of(0))

In [142]:
train = np.vstack([train,x.raw_image.reshape((1,32,32,1))])
labels = np.vstack([labels, x.label.reshape((1,10))])

In [165]:
print(train.shape)
print(labels.shape)

(1, 32, 32, 1)
(1, 10)


In [73]:
from tensorflow import stack

In [76]:
x.raw_image.shape
x.label.shape

(10,)

In [145]:
model.fit(train,labels)

1/1 [==============================] - 0s 18ms/step - loss: 2.2975 - accuracy: 0.0000e+00


In [172]:
def get_data_split(file="training_data", image_retriever = ImageRetriever()):
    path = os.path.join("Dataset",file)
    train = np.ndarray((0,32,32,1))
    labels = np.ndarray((0,9))
    for number_folder in os.listdir(path):
        for number_image in os.listdir(os.path.join(path,number_folder)):
            image_path = os.path.join(path,number_folder,number_image)
            image_retriever.get(image_path)
            train = np.vstack([train,image_retriever.raw_image.reshape((1,32,32,1))])
            labels = np.vstack([labels, image_retriever.label.reshape((1,9))])
    return train, labels
            
    

train,labels = get_data_split()

In [203]:
model.fit(train,labels, epochs = 5)

Epoch 1/5
180/180 [==============================] - 3s 14ms/step - loss: 0.4551 - accuracy: 0.8649
Epoch 2/5
180/180 [==============================] - 3s 14ms/step - loss: 0.0511 - accuracy: 0.9866
Epoch 3/5
180/180 [==============================] - 3s 14ms/step - loss: 0.0277 - accuracy: 0.9920
Epoch 4/5
180/180 [==============================] - 3s 15ms/step - loss: 0.0140 - accuracy: 0.9958
Epoch 5/5
180/180 [==============================] - 3s 14ms/step - loss: 0.0107 - accuracy: 0.9967


In [205]:
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 12, 12, 16)        4624      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 6, 6, 16)         0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 576)               0         
                                                                 
 dense_14 (Dense)            (None, 64)               

In [193]:
test, test_labels = get_data_split(file = "testing_data")

In [206]:
print(model.evaluate(test,test_labels))

31/31 [==============================] - 0s 4ms/step - loss: 3.8148e-04 - accuracy: 1.0000
[0.000381484191166237, 1.0]


In [195]:
x.get(get_rand_pic_of(4))

In [199]:
image = x.raw_image
model.predict(image.reshape((1,32,32,1)))
results = model.predict(image.reshape((1,32,32,1)))
results = np.argmax(results,axis = 1)

In [200]:
print(results)

[4]
